<a href="https://colab.research.google.com/github/CG0718/Documentation-Generator-agent/blob/main/using_ScaleDownHaste.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import subprocess

# Ensure a clean slate: remove any existing 'scaledown' directory before cloning
if os.path.exists('scaledown'):
    print("Removing existing 'scaledown' directory...")
    subprocess.run(['rm', '-rf', 'scaledown'], check=True)
    print("'scaledown' directory removed.")

# Clone the repository
print("Cloning scaledown repository...")
!git clone https://github.com/scaledown-team/scaledown.git
print("Repository cloned.")

# Change directory into the cloned repository to perform the installation
%cd scaledown

# Create a virtual environment (optional, but good practice)
!python -m venv .venv

# Install the package in editable mode with 'haste' and 'semantic' extras
# This makes the package discoverable by Python without needing to modify sys.path directly.
print("Installing scaledown in editable mode...")
!pip install -e ".[haste,semantic]"
print("scaledown installed.")

# Change back to the original content directory
%cd ..
print("Returned to original directory.")

In [ ]:
!export SCALEDOWN_API_KEY="SCALEDOWN_API_KEY"
!export SCALEDOWN_API_URL="https://api.scaledown.xyz"  # Optional, uses default if not set
!export GITHUB_FILE="FILE_PATH"

In [ ]:
from scaledown.optimizer import HasteOptimizer
import os

api_key = os.getenv("SCALEDOWN_API_KEY", "")

optimizer = HasteOptimizer(api_key=api_key, top_k=5, semantic=False)


result = optimizer.optimize(
    context="",# Can be empty when file_path is provided
    query="explain the all main code logic blocks",
    file_path="GITHUB_FILE"
)
print(result.content)  # Optimized code
print(f"Compression: {result.metrics.compression_ratio:.2f}x")
with open("result.md", "w") as f:
            f.write(result.content)

In [ ]:
!pip install -q -U google-generativeai
!export YOUR_GEMINI_API_KEY="YOUR_GEMINI_API_KEY"

In [ ]:
import google.generativeai as genai
import os

genai.configure(api_key="YOUR_GEMINI_API_KEY")

def generate_markdown_docs(source_file_path, output_md_path):
    model = genai.GenerativeModel('gemini-1.5-flash')

    try:
        with open(source_file_path, 'r') as file:
            source_code = file.read()
    except FileNotFoundError:
        print(f"Error: {source_file_path} not found.")
        return

    prompt = f"""
    Act as a professional technical writer and generate a comprehensive documentation for the following code which is easily understandable.

    Code to document:
    ```
    {source_code}
    ```
    """

    print(f"Generating documentation for {source_file_path}...")
    response = model.generate_content(prompt)

    with open(output_md_path, 'w') as md_file:
        md_file.write(response.text)

    print(f"Successfully saved documentation to: {output_md_path}")


if __name__ == "__main__":
    generate_markdown_docs('result.md', 'DOCUMENTATION.md')